In [35]:
from flask import Flask, render_template_string, request, jsonify
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import requests
import numpy as np
from collections import defaultdict
!pip install pyngrok
from pyngrok import ngrok

app = Flask(__name__)

# SHL API configuration (replace with actual SHL API details)
SHL_API_URL = "https://api.shl.com/assessments"  # Example URL
API_KEY = "your_shl_api_key_here"  # Should be stored securely in production

# Mock data for demonstration (same as before)
def get_shl_assessments():
    """Fetch SHL assessments from their API or use mock data"""
    return [
        {
            "id": "1",
            "name": "Verify Interactive",
            "description": "Cognitive ability test measuring verbal, numerical, and inductive reasoning",
            "skills": ["cognitive", "verbal", "numerical", "reasoning"],
            "job_level": ["entry", "mid"],
            "duration": 45,
            "popularity": 8.5
        },
        {
                "id": "2",
                "name": "OPQ32",
                "description": "Personality questionnaire measuring behavioral preferences at work",
                "skills": ["personality", "behavior", "work_preferences"],
                "job_level": ["all"],
                "duration": 30,
                "popularity": 9.0
            },
            {
                "id": "3",
                "name": "SJT Professional",
                "description": "Situational Judgment Test for professional roles",
                "skills": ["judgment", "decision_making", "professional_skills"],
                "job_level": ["mid", "senior"],
                "duration": 25,
                "popularity": 7.8
            },
            {
                "id": "4",
                "name": "Motivational Questionnaire",
                "description": "Measures what drives and motivates individuals at work",
                "skills": ["motivation", "drivers", "engagement"],
                "job_level": ["all"],
                "duration": 20,
                "popularity": 7.2
            },
            {
                "id": "5",
                "name": "Deductive Reasoning",
                "description": "Measures logical reasoning and problem-solving skills",
                "skills": ["cognitive", "logical_reasoning", "problem_solving"],
                "job_level": ["entry", "mid", "senior"],
                "duration": 35,
                "popularity": 8.1
            }
        ]

def preprocess_data(assessments):
    """Convert assessments data to a format suitable for recommendation"""
    processed = []
    for assessment in assessments:
        processed.append({
            "id": assessment["id"],
            "content": f"{assessment['name']} {assessment['description']} {' '.join(assessment['skills'])} {' '.join(assessment['job_level'])}"
        })
    return processed

# Recommendation engine
class SHLRecommender:
    def __init__(self):
        self.assessments = get_shl_assessments()
        self.processed_data = preprocess_data(self.assessments)
        self.vectorizer = TfidfVectorizer(stop_words='english')
        self.tfidf_matrix = self.vectorizer.fit_transform([item['content'] for item in self.processed_data])

    def recommend_by_query(self, query, top_n=3):
        """Recommend assessments based on text query"""
        query_vec = self.vectorizer.transform([query])
        similarities = cosine_similarity(query_vec, self.tfidf_matrix).flatten()
        related_indices = similarities.argsort()[::-1][:top_n]

        recommendations = []
        for idx in related_indices:
          recommendations.append({
    "assessment": self.assessments[idx],  # Corrected to match variable name
    "similarity_score": float(similarities[idx])
          })


        return recommendations

    def recommend_by_job_level(self, job_level, top_n=3):
        """Recommend assessments based on job level"""
        matched = [a for a in self.assessments if job_level in a['job_level']]
        matched_sorted = sorted(matched, key=lambda x: x['popularity'], reverse=True)
        return matched_sorted[:top_n]

    def hybrid_recommendation(self, query=None, job_level=None, top_n=3):
        """Combine content-based and popularity-based recommendations"""
        if query and job_level:
            content_recs = self.recommend_by_query(query, top_n*2)
            level_recs = self.recommend_by_job_level(job_level, top_n*2)

            # Combine and deduplicate
            combined = content_recs + [{"assessment": a, "similarity_score": 0} for a in level_recs]
            unique_recs = {}
            for rec in combined:
                aid = rec['assessment']['id']
                if aid not in unique_recs:
                    unique_recs[aid] = rec
                else:
                    # Boost score if present in both
                    unique_recs[aid]['similarity_score'] += 0.5

            # Sort by combined score
            sorted_recs = sorted(unique_recs.values(),
                               key=lambda x: (x['similarity_score'], x['assessment']['popularity']),
                               reverse=True)
            return [r['assessment'] for r in sorted_recs[:top_n]]

        elif query:
            return [r['assessment'] for r in self.recommend_by_query(query, top_n)]
        elif job_level:
            return self.recommend_by_job_level(job_level, top_n)
        else:
            return sorted(self.assessments, key=lambda x: x['popularity'], reverse=True)[:top_n]

# Evaluation metrics
class EvaluationMetrics:
    @staticmethod
    def precision_at_k(recommendations, relevant_items, k):
        """Calculate precision at K"""
        top_k = recommendations[:k]
        relevant_set = set(relevant_items)
        relevant_and_retrieved = [item for item in top_k if item['id'] in relevant_set]
        return len(relevant_and_retrieved) / k

    @staticmethod
    def recall_at_k(recommendations, relevant_items, k):
        """Calculate recall at K"""
        top_k = recommendations[:k]
        relevant_set = set(relevant_items)
        relevant_and_retrieved = [item for item in top_k if item['id'] in relevant_set]
        return len(relevant_and_retrieved) / len(relevant_items) if relevant_items else 0

    @staticmethod
    def mean_average_precision(recommendations, relevant_items):
        """Calculate mean average precision"""
        relevant_set = set(relevant_items)
        average_precision = 0.0
        num_relevant = 0

        for k in range(1, len(recommendations)+1):
            if recommendations[k-1]['id'] in relevant_set:
                num_relevant += 1
                precision_at_k = num_relevant / k
                average_precision += precision_at_k

        return average_precision / len(relevant_items) if relevant_items else 0

    @staticmethod

    def evaluate(recommender, test_cases):
        """Evaluate the recommender system with multiple test cases"""
        results = {
            'precision@3': [],
            'recall@3': [],
            'map': []
        }

        for case in test_cases:
            query = case.get('query')
            job_level = case.get('job_level')
            relevant_ids = case['relevant_ids']

            recommendations = recommender.hybrid_recommendation(query, job_level, top_n=5)
            count = 0
            recall_score = count/len(relevant_ids)

            results['precision@3'].append(
                EvaluationMetrics.precision_at_k(recommendations, relevant_ids, 3)
            )
            results['recall@3'].append(
                EvaluationMetrics.recall_at_k(recommendations, relevant_ids, 3)
            )
            results['map'].append(
                EvaluationMetrics.mean_average_precision(recommendations, relevant_ids)
            )

        # Calculate averages
        avg_metrics = {
            'avg_precision@3': sum(results['precision@3']) / len(results['precision@3']),
            'avg_recall@3': sum(results['recall@3']) / len(results['recall@3']),
            'avg_map': sum(results['map']) / len(results['map'])
        }

        return avg_metrics

# Create test cases for evaluation
def create_test_cases(assessments):
    """Create test cases for evaluation based on assessment data"""
    # In a real scenario, these would come from user testing or historical data
    test_cases = [
        {
            "query": "cognitive reasoning test",
            "job_level": "entry",
            "relevant_ids": ["1", "5"]  # Verify Interactive and Deductive Reasoning
        },
        {
            "query": "personality questionnaire",
            "job_level": "all",
            "relevant_ids": ["2", "4"]  # OPQ32 and Motivational Questionnaire
        },
        {
            "query": "professional skills assessment",
            "job_level": "senior",
            "relevant_ids": ["3"]  # SJT Professional
        },
        {
            "query": "logical reasoning",
            "job_level": "mid",
            "relevant_ids": ["1", "5"],  # Verify Interactive and Deductive Reasoning
            "description": "Test for logical reasoning assessments"
        },
        {
            "query": "work motivation",
            "job_level": "all",
            "relevant_ids": ["4"],  # Motivational Questionnaire
            "description": "Test for motivation assessments"
        }
    ]


    return test_cases

# Modified HTML template as a string (since we can't use templates folder in Colab)
HTML_TEMPLATE = """
<!DOCTYPE html>
<html>
<head>
    <title>SHL Assessment Recommender</title>
    <style>
        body { font-family: Arial, sans-serif; max-width: 800px; margin: 0 auto; padding: 20px; }
        .container { display: flex; flex-direction: column; gap: 20px; }
        .input-group { display: flex; flex-direction: column; gap: 5px; }
        input, select, button { padding: 8px; font-size: 16px; }
        button { background-color: #007bff; color: white; border: none; cursor: pointer; padding: 10px; }
        button:hover { background-color: #0056b3; }
        .results { margin-top: 20px; border-top: 1px solid #ddd; padding-top: 20px; }
        .assessment-card { border: 1px solid #ddd; padding: 15px; margin-bottom: 10px; border-radius: 5px; }
        .metrics { background-color: #f8f9fa; padding: 15px; border-radius: 5px; margin-top: 30px; }
    </style>
</head>
<body>
    <div class="container">
        <h1>SHL Assessment Recommender</h1>

        <div class="input-group">
            <label for="query">What skills or attributes are you looking to assess?</label>
            <input type="text" id="query" placeholder="e.g. cognitive skills, personality, etc.">
        </div>

        <div class="input-group">
            <label for="job_level">Job Level:</label>
            <select id="job_level">
                <option value="">Any level</option>
                <option value="entry">Entry Level</option>
                <option value="mid">Mid Level</option>
                <option value="senior">Senior Level</option>
            </select>
        </div>

        <button onclick="getRecommendations()">Get Recommendations</button>

        <div class="results" id="results">
            <!-- Recommendations will appear here -->
        </div>

        <div class="metrics">
            <h2>System Performance</h2>
            <p>Our recommendation engine achieves the following evaluation metrics:</p>
            <ul id="metrics-list">
                <li>Loading metrics...</li>
            </ul>
            <button onclick="refreshMetrics()">Refresh Metrics</button>
        </div>
    </div>

    <script>
        async function getRecommendations() {
            const query = document.getElementById('query').value;
            const job_level = document.getElementById('job_level').value;

            const response = await fetch('/recommend', {
                method: 'POST',
                headers: {
                    'Content-Type': 'application/json',
                },
                body: JSON.stringify({
                    query: query,
                    job_level: job_level
                })
            });

            const data = await response.json();
            displayResults(data.recommendations);
        }

        function displayResults(recommendations) {
            const resultsDiv = document.getElementById('results');

            if (!recommendations || recommendations.length === 0) {
                resultsDiv.innerHTML = '<p>No recommendations found. Try a different query.</p>';
                return;
            }

            let html = '<h2>Recommended Assessments</h2>';

            recommendations.forEach(assessment => {
                html += `
                <div class="assessment-card">
                    <h3>${assessment.name}</h3>
                    <p>${assessment.description}</p>
                    <p><strong>Skills assessed:</strong> ${assessment.skills.join(', ')}</p>
                    <p><strong>Job levels:</strong> ${assessment.job_level.join(', ')}</p>
                    <p><strong>Duration:</strong> ${assessment.duration} minutes</p>
                </div>
                `;
            });

            resultsDiv.innerHTML = html;
        }

        async function loadMetrics() {
            const response = await fetch('/evaluate');
            const data = await response.json();
            const metrics = data.evaluation_metrics;

            const metricsList = document.getElementById('metrics-list');
            metricsList.innerHTML = `
                <li><strong>Precision@3:</strong> ${(metrics.avg_precision_at_3 * 100).toFixed(1)}%</li>
                <li><strong>Recall@3:</strong> ${(metrics.avg_recall_at_3 * 100).toFixed(1)}%</li>
                <li><strong>Mean Average Precision:</strong> ${(metrics.avg_map * 100).toFixed(1)}%</li>
            `;
        }

        function refreshMetrics() {
            loadMetrics();
        }

        // Load metrics when page loads
        window.onload = loadMetrics;
    </script>
</body>
</html>
"""

@app.route('/')
def home():
    return render_template_string(HTML_TEMPLATE)

# ... [Keep all the other routes from the original code: /recommend, /evaluate]

@app.route('/recommend', methods=['POST'])
def recommend():
    try:
        data = request.get_json()
        query = data.get('query')
        job_level = data.get('job_level')

        recommender = SHLRecommender()
        recommendations = recommender.hybrid_recommendation(query, job_level)

        return jsonify({
            "status": "success",
            "recommendations": recommendations
        })
    except Exception as e:
        return jsonify({
            "status": "error",
            "message": str(e)
        }), 500
@app.route('/evaluate')
def evaluate():
    recommender = SHLRecommender()
    test_cases = create_test_cases(recommender.assessments)
    metrics = EvaluationMetrics.evaluate(recommender, test_cases)

    return jsonify({
        "status": "success",
        "evaluation_metrics": metrics
    })

# @app.route('/evaluate', methods=['GET'])
# def evaluate():
#     """Endpoint to run evaluation and return metrics"""
#     try:
#         recommender = SHLRecommender()
#         test_cases = create_test_cases(recommender.assessments)
#         metrics = EvaluationMetrics.evaluate(recommender, test_cases)

#         # Format results for better display
#         formatted_metrics = {
#             "Precision@3": f"{metrics['avg_precision@3']*100:.1f}%",
#             "Recall@3": f"{metrics['avg_recall@3']*100:.1f}%",
#             "Precision@5": f"{metrics['avg_precision@5']*100:.1f}%",
#             "Recall@5": f"{metrics['avg_recall@5']*100:.1f}%",
#             "MAP": f"{metrics['mean_average_precision']*100:.1f}%",
#             "MRR": f"{metrics['mean_reciprocal_rank']*100:.1f}%",
#             "Test Cases": len(test_cases)
#         }

#         return jsonify({
#             "status": "success",
#             "metrics": formatted_metrics,
#             "raw_metrics": metrics  # For debugging
#         })
#     except Exception as e:
#         return jsonify({
#             "status": "error",
#             "message": str(e)
#         }), 500


# if __name__ == '__main__':
#     # Install ngrok if not already installed
#     !pip install pyngrok

#     # Start ngrok tunnel


#     from pyngrok import ngrok
#     public_url = ngrok.connect(5000).public_url
#     print(f" * Running on {public_url}")

#     # Run the app
#     app.run()
import subprocess

if __name__ == '__main__':
    # Install pyngrok
    subprocess.run(['pip', 'install', 'pyngrok'], check=True)

    # Set your ngrok authtoken (replace with your actual token)
    NGROK_AUTH_TOKEN = "2waVPVr8AuX0MeP60cxIWdiUas3_GFBUuUg3tFuJ65oPHgQu"  # e.g., "2AbC...xyz"
    ngrok.set_auth_token(NGROK_AUTH_TOKEN)

    try:
        # Start ngrok tunnel
        public_url = ngrok.connect(5000).public_url
        print(f" * Running on {public_url}")
        print(" * If the URL gives 502, wait a minute and refresh")

        # Run the app
        app.run()
    except Exception as e:
        print(f"Error: {e}")
        print("Running without ngrok (only accessible within Colab)")
        app.run()

 * Running on https://865e-34-73-44-227.ngrok-free.app
 * If the URL gives 502, wait a minute and refresh
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [04/May/2025 16:17:41] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [04/May/2025 16:17:42] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [04/May/2025 16:17:42] "GET /evaluate HTTP/1.1" 200 -
